In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import Adam

In [3]:
print(tf.__version__)

2.13.0


In [4]:
data = pd.read_csv('C:/Users/Ihab/Desktop/PCyber/spam.csv')
texts = data['Message'].tolist()
labels = data['Category'].tolist()

In [5]:
enc = {'ham': 1, 'spam': 0}
data['Category'] = data['Category'].map(enc)

In [6]:
texts = data['Message'].tolist()
labels = data['Category'].tolist()

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\users\ihab\appdata\local\programs\python\python38\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ihab\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [8]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [9]:
for fold, (train_idx, val_idx) in enumerate(kf.split(texts)):
    print(f"Training fold {fold+1}/{kf.get_n_splits()}")
    
    # Split data for the current fold
    X_train, X_val = [texts[i] for i in train_idx], [texts[i] for i in val_idx]
    y_train, y_val = [labels[i] for i in train_idx], [labels[i] for i in val_idx]

Training fold 1/5
Training fold 2/5
Training fold 3/5
Training fold 4/5
Training fold 5/5


In [10]:
train_encodings = tokenizer(
        X_train,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="tf"
    )

val_encodings = tokenizer(
        X_val,
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors="tf"
    )


In [11]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model.compile(optimizer=Adam(learning_rate=2e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [14]:
train_dataset = (
        tf.data.Dataset.from_tensor_slices((dict(train_encodings), y_train))
        .shuffle(1000)
        .batch(16)
        .cache()
        .prefetch(tf.data.AUTOTUNE)
    )

val_dataset = (
        tf.data.Dataset.from_tensor_slices((dict(val_encodings), y_val))
        .batch(16)
        .cache()
        .prefetch(tf.data.AUTOTUNE)
    )

In [ ]:
model.fit(train_dataset, validation_data=val_dataset, epochs=3)

In [ ]:
accuracies = []


    val_loss, val_accuracy = model.evaluate(val_dataset)
    accuracies.append(val_accuracy)

print(f"Mean Accuracy after K-Folds: {sum(accuracies)/len(accuracies)}")
